In [1]:
import sys
sys.path.insert(0, '/home/llr/cms/wind/.local/lib/python2.7/site-packages/')

In [2]:
from bayes_opt import BayesianOptimization
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import gridspec

In [3]:
from subprocess import check_output

In [4]:
from trainlib.ConfigFileHandler import ConfigFileHandler

In [5]:
run_dir = "/data_CMS/cms/wind/180411_class_weights_Adam_optimized_softjets/input_columns_4/"
engine = "tree"
out_dir = "/data_CMS/cms/wind/prior_evaluator/"

In [6]:
def punzi_target(ggH_prior, WHhadr_prior, ZHhadr_prior, WHlept_prior, ZHlept_prior, ZHMET_prior, ttHhadr_prior, ttHlept_prior):
    bin_dir = "/home/llr/cms/wind/cmssw/CMSSW_9_4_2/bin/slc6_amd64_gcc630/"
    cost_function_evaluator = "run_prior_evaluator"
    
    #output = check_output([bin_dir + cost_function_evaluator, run_dir, out_dir, engine, str(ggH_prior), str(WHh_prior), 
    #          str(ZHh_prior), str(WHl_prior), str(ZHl_prior), str(ZHMET_prior), str(ttHh_prior), str(ttHl_prior)])
    
    #costval = 0.0

    #for line in output.split('\n'):
    #    if "cost = " in line:
    #        costval = -float(line.replace("cost = ", ""))
    #        break
    
    costval = -(ggH_prior - 0.5)**2 - (WHhadr_prior - 0.1)**2 - (ZHhadr_prior - 0.9)**2- (WHlept_prior - 0.1)**2 + ZHMET_prior - 0.5 * ttHhadr_prior - 0.134 * ttHlept_prior
     
    return costval

In [7]:
eps = 1e-3

In [8]:
bo = BayesianOptimization(punzi_target, {'ggH_prior': (1.0, 4.0), 'WHhadr_prior': (eps, 1.0), 
                                   'ZHhadr_prior': (eps, 1.0), 'WHlept_prior': (eps, 1.0),
                                   'ZHlept_prior': (eps, 1.0), 'ZHMET_prior': (eps, 1.0),
                                   'ttHhadr_prior': (eps, 1.0), 'ttHlept_prior': (eps, 1.0)})

In [ ]:
bo.maximize(init_points = 60, n_iter = 0, acq = 'ucb', kappa = 5)

Initialization
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   WHhadr_prior |   WHlept_prior |   ZHMET_prior |   ZHhadr_prior |   ZHlept_prior |   ggH_prior |   ttHhadr_prior |   ttHlept_prior | 
    1 | 00m00s |   -6.36579 |         0.5989 |         0.5184 |        0.2251 |         0.8505 |         0.9130 |      2.9453 |          0.2813 |          0.3300 | 
    2 | 00m00s |   -1.59160 |         0.5845 |         0.7512 |        0.7421 |         0.5233 |         0.4075 |      1.5576 |          0.6419 |          0.6975 | 
    3 | 00m00s |  -10.89353 |         0.3335 |         0.7842 |        0.2103 |         0.0116 |         0.3528 |      3.6186 |          0.0630 |          0.2581 | 
    4 | 00m00s |  -10.29982 |         0.8208 |         0.6480 |        0.0898 |         0.8704 |         0.5088 |      3.5488 |          0.5283 |          0.0709

In [ ]:
for it in range(5):
    bo.maximize(init_points = 60, n_iter = 60, acq = 'ucb', kappa = 5)

Bayesian Optimization
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   WHhadr_prior |   WHlept_prior |   ZHMET_prior |   ZHhadr_prior |   ZHlept_prior |   ggH_prior |   ttHhadr_prior |   ttHlept_prior | 
   61 | 00m06s |    0.71976 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         0.0010 |      1.0000 |          0.0010 |          0.0010 | 
   62 | 00m06s |    0.58590 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          1.0000 | 
   63 | 00m07s |    0.08640 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         0.0010 |      1.0000 |          1.0000 |          1.0000 | 
   64 | 00m06s |    0.22026 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          1.0000 |         

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -2.46165992e-05]), 'nit': 4, 'funcalls': 47}
  " state: %s" % convergence_dict)


   92 | 00m30s |    0.33157 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.5698 |          0.7758 | 
   93 | 00m30s |    0.62557 |         0.2933 |         0.0010 |        1.0000 |         1.0000 |         0.0010 |      1.0000 |          0.0010 |          0.4983 | 
   94 | 00m31s |    0.30882 |         0.3510 |         0.5834 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          1.0000 | 
   95 | 00m29s |    0.64945 |         0.0010 |         0.2950 |        1.0000 |         0.7650 |         1.0000 |      1.0000 |          0.0010 |          0.2537 | 
   96 | 00m32s |    0.66601 |         0.2896 |         0.2934 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          0.0010 | 
   97 | 00m34s |    0.49665 |         0.2736 |         0.0010 |        1.0000 |         0.7451 |         1.0000 |      1.0000 |          0.3786 |          0.0010 | 
   98 | 00

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -9.02500935e-05]), 'nit': 6, 'funcalls': 57}
  " state: %s" % convergence_dict)


  134 | 01m38s |    0.72716 |         0.1490 |         0.0010 |        1.0000 |         1.0000 |         0.3749 |      1.0000 |          0.0010 |          0.0010 | 
  135 | 01m40s |    0.74648 |         0.1228 |         0.0515 |        1.0000 |         0.9038 |         0.0025 |      1.0000 |          0.0010 |          0.0010 | 


...

  154 | 03m45s |    0.72932 |         0.0010 |         0.0010 |        1.0000 |         0.9211 |         0.0779 |      1.0000 |          0.0010 |          0.0010 | 
  155 | 04m05s |    0.74838 |         0.1143 |         0.1144 |        1.0000 |         0.8761 |         1.0000 |      1.0000 |          0.0010 |          0.0010 | 
  156 | 03m55s |    0.69455 |         0.0010 |         0.1649 |        1.0000 |         1.0000 |         0.1110 |      1.0000 |          0.0010 |          0.2308 | 


/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00011339]), 'nit': 6, 'funcalls': 51}
  " state: %s" % convergence_dict)


  157 | 04m22s |    0.72101 |         0.0010 |         0.1925 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          0.0010 | 


/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([  9.29640346e-05]), 'nit': 6, 'funcalls': 49}
  " state: %s" % convergence_dict)


  158 | 04m20s |    0.72956 |         0.0010 |         0.1015 |        1.0000 |         1.0000 |         0.4509 |      1.0000 |          0.0010 |          0.0010 | 
  159 | 04m42s |    0.72210 |         0.0521 |         0.0010 |        1.0000 |         0.7768 |         0.1338 |      1.0000 |          0.0010 |          0.0010 | 
